In [15]:
import os
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import time
import requests
import random
import sqlite3
import string
import Image
import pytesseract
from PIL import Image
from datetime import datetime

In [16]:
class Complaint:
    
    meses = [u'Janeiro', u'Fevereiro', u'Março', u'Abril', u'Maio', 
             u'Junho', u'Julho', u'Agosto', u'Setembro', u'Outubro', 
             u'Novembro', u'Dezembro']

    def __init__(self):
        self.complaintID = None
        self.business = None
        self.status = None
        self.complaint = None
        self.businessURL = None
    
    def setPlace(self, rawPlace):
        self.place = clrPlace(rawPlace)
    
    def setDate(self, arg):
        self.Date = clrDate(rawDate)
    
    def clrPlace(arg):
        arg = arg.split(',')[2].strip()
        return arg
    
    def clrDate(arg):
        match = setDate(arg.split(',')[1].strip())
        arg = time.strptime(match.group().replace(' ',''), '%d-%m-%Y-%H:%M')
        return arg
    
    def clearDate(arg):
        try:
            for i in range(12):
                arg = arg.replace(meses[i],u'0'+str(i+1))
                arg = arg.replace(u'de', '-')
                arg = re.search(r'\d{2} - \d{2} - \d{4} - \d{2}:\d{2}', txt)
            return arg
        except:
            print 'Is not a String object'

In [17]:
class Database:
    
    def __init__(self, name):
        self.db_name = name
    
    def setUser(self, user):
        self.user = user
    
    def setHost(self, host):
        self.host = host
        
    def setPass(self, password):
        self.pswrd = password

In [18]:
class ConnectionFactory:                                       
    def getConnection(self):
        try:
            self.connection = self.sgbd.connect(self.dbAtrib)
            return self.connection
        except:
            print "I am unable to connect to the database"

In [19]:
class Sqlite(ConnectionFactory):
    def __init__(self, database):
        self.db = database
        self.sgbd = sqlite3
        self.dbAtrib = db.db_name

In [20]:
class PostgreSql(ConnectionFactory):
    def __init__(self, database):
        self.db = database
        self.sgbd = sqlite3
        self.dbAtrib = ("dbname='"+self.db_name+"' user='"+self.user+"' host='"
                       +self.localhost+"' password='"+self.pswrd+"'")

In [21]:
class Dao:
    
    def __init__(self,sgbd):
        self.sgbd = sgbd
        self.connection = sgbd.getConnection()
        self.cursor = self.connection.cursor()
        self.kindDao = None
        self.att = None
        self.setAtrib()   
      
    def setAtrib(self): 
        pass
    
    def createTable(self, table = None, att = None):
        try:
            if table is None or att is None:
                table = self.kindDao
                att = self.att
            self.query("CREATE TABLE "+table+" "+att+";")
            self.commit()
        
        except:
            print "Could not create the table"
    
    def select(self, arg, table = None):
        try:
            if table is None:
                table = self.kindDao
            qry= ("SELECT "+arg+" FROM "+table+";")
            self.query(qry)
            self.commit()
            return self.cursor.fetchall()
        except:
            print "Select error, verify your arguments"
    
    def putQuery(self, query):
        try:
            self.query(query)
            self.commit()
            return self.cursor.fetchall()
        except:
            print "Select error, verify your arguments"
        
    def insert(self, arg):
        qry = self.setQry(arg)
        self.query(qry)
        self.commit()
    
    def delete(self, atrib, cond, val):
        self.query("DELETE FROM "+self.kindDao+" "+ "WHERE "+atrib+cond+"'"+val+"';")
        self.commit()
    
    def drop(self):
        self.query("DROP TABLE "+self.kindDao+";")
        self.commit()
        
    def commit(self):
        self.connection.commit()
        
    def query(self, arg):
        self.cursor.execute(arg)
        
    def setQry(self, arg):
        pass
    
    def getTables(self):
        qry= ("select name from sqlite_master where type = 'table';")
        self.query(qry)
        self.commit()
        return self.cursor.fetchall()

In [22]:
class RankDao(Dao):
    def setAtrib(self):
        self.kindDao = "rank"
        self.att = "(rank_ID INTEGER PRIMARY KEY AUTOINCREMENT,business TEXT, url TEXT)" 
    def setQry(self, arg):
        return "INSERT INTO "+self.kindDao+" (business, url) VALUES ('"+arg.business+"', '"+arg.businessURL+"');"

In [23]:
class AddressDao(Dao):
    def setAtrib(self):
        self.kindDao = "address"
        self.att = ("(add_ID INTEGER PRIMARY KEY AUTOINCREMENT, business TEXT,"
                    "status TEXT, url TEXT, place TEXT, date TIME)")
    
    def setQry(self, arg):
        return ("INSERT INTO "+self.kindDao+" (business,"
                    "status, url, place, date TIME) VALUES ('"+arg.business+"', '"
                           +arg.status+"', '"+arg.businessURL+'", "'+arg.place+'", "'+arg.date+"');")

In [107]:
class complaintDao(Dao):
    def setAtrib(self):
        self.kindDao = "complaint"
        self.att = ("(compl_ID INTEGER PRIMARY KEY AUTOINCREMENT, business TEXT, status TEXT"
        ", url TEXT, place TEXT, date TEXT, complaint TEXT)")
    
    def setQry(self, arg):
        return ("INSERT INTO "+self.kindDao+" (business, status"
        ", url, place, date, complaint) VALUES ('"+arg.business+"', '"
                +arg.status+"', '"+arg.businessURL+"', '"+arg.place+"', '"+arg.date+"', '"+arg.complaint+"');")

In [108]:
class WdChain:
    
    dao = None
    
    def __init__(self,webdriver,dao):
        self.wd = webdriver
        self.driver = webdriver.Firefox()
        self.dao = dao
        self.nextWd = None
        self.income = None
        self.complaint = Complaint()
        
    def setNext(self, link):
        if self.nextWd == None:
            self.nextWd = link;
        else: 
            self.nextWd.setNext(forma);    
    
    def scrap(self, arg):
        self.income = WebScrap(arg)
        if income is "error":
            if self.nextWd is None:
                self.setLog()
                return "None WebScrapper implemented for this HTML"
            else:
                self.nextWd.scrap(arg)
                
    def WebScrap(self):
        pass
    
    def setLog(self):
        if (u'complaint',) in self.dao.getTables():
            args = [4]
            args[0] = datetime.utcnow().ctime()
            args[1] = u'complaint'
            args[2] = self.dao.putQuery("SELECT business FROM complaint WHERE"
                                         "compl_ID = (SELECT MAX(compl_ID) FROM complaint);")[0][0]
            args[3] = self.dao.putQuery("SELECT COUNT(compl_ID) FROM complaint GROUP BY business HAVING"
                         "compl_ID = (SELECT MAX(compl_ID) FROM complaint);")[0][0]
            args[4] = self.dao.putQuery("SELECT MAX(compl_ID) FROM complaint;")[0][0]
            
            self.commit(args)
        elif (u'address',) in dao.getTables():
            args = [4]
            args[0] = datetime.utcnow().ctime()
            args[1] = u'address'
            args[2] = self.dao.putQuery("SELECT business FROM address WHERE"
                                         "add_ID = (SELECT MAX(add_ID) FROM address);")[0][0]
            args[3] = self.dao.putQuery("SELECT COUNT(add_ID) FROM address GROUP BY business HAVING"
                         "add_ID = (SELECT MAX(add_ID) FROM address);")[0][0]
            args[4] = self.dao.putQuery("SELECT MAX(add_ID) FROM address;")[0][0]
            
            self.commit(args)
        else:
            pass
        
    def commit(self, args):
        self.dao.putQuery("INSERT INTO log (time,tableName, business, quant, lastIN_ID)"
                          " VALUES ('"+args[0]+"', '"+args[1]+"', '"+args[2]+"', "+arg[3]+", "+arg[4]+");")

In [26]:
class ScrapRank(WdChain):
    
    def WebScrap(self):
        try: 
            if self.dao.kindDao is not "rank":
                raise ValueError('erro')
            for i in range(20):
                self.complaint.business = self.driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/'
                            'td/div/table['+str(i+1)+']/tbody/tr/td/a').get_attribute("title")
                self.complaint.businessURL = self.driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/td/'
                            'div/table['+str(i+1)+']/tbody/tr/td/a').get_attribute("href")
                self.dao.insert(self.complaint)
            self.dao = AddressDao(self.dao.sgbd)
            raise ValueError('error')
        except:
            return "error"

In [109]:
# FIX THIS MESS AS SOON AS POSSIBLE
class ScrapAddress(WdChain):
    def WebScrap(self):
        try:
            last_rank_id = lastLog_id()
            last_business = self.dao.putQuery("SELECT business, url FROM rank WHERE rank_id > "+last_rank_id+";")
            firstLevelScrap(self.last_business)
            self.dao = complaintDao(self.dao.sgbd)
        except:
            return "error"
        
    def firstLevelScrap(self, last_business):
        try:
            for busUrl in last_business:
                        self.driver.get(busUrl[1])
                        #time.sleep(5)
                        key = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"'
                                                           ']/div/div[2]/div/div/ul/li/a').get_attribute("href")
                        self.driver.get(key)
                        secondLevelScrap(self,busUrl[0])
                        
        except:
            return "error"
        
    def secondLevelScrap(self, business):    
        try:
            count = 0
            while count<=40000:
                self.complaint.businessID = 0
                addChain = WdChain()
                nextPage = driver.find_element_by_xpath('//*[@id="todo-conteudo-resultado"]/div/div/center/div/ul/li[7]/a').get_attribute("href")
                selfdriver.get(nextPage)
                cout+=1
        except:
            raise ValueError('error')

In [28]:
class ScrapAddrss(WdChain):
    def WebScrap(self):
        try:
            for j in range(20):
                self.complaint.business = business
                self.complaint.businessURL  =  link = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"'
                    ']/div/div[2]/div/div[2]/ul/li[' + str(j+1) +']/h3/a').get_attribute("href") 
                self.dao.insert(self.complaint)
        except:
            return "error"        

In [29]:
class BreakCaptchaLinux(WdChain):
    def WebScrap(self):
        try:
            oldtab = self.wd.current_window_handle
            body = self.wd.find_element_by_tag_name("body")
            body.send_keys(Keys.CONTROL + 't')
            self.wd.get('http://www.reclameaqui.com.br/indices/lista_reclamacoes/captcha.php')
            #You have to create the folder /captcha
            self.wd.save_screenshot("../RI/captcha/screenshot.png")
            self.wd.find_element_by_tag_name("body").send_keys(Keys.ALT + Keys.NUMPAD1)
            self.wd.switch_to_window(oldtab)
            captcha = pytesseract.image_to_string(Image.open('../RI/captcha/screenshot.png'), config="-psm 6")
            cp = self.driver.find_element_by_name('captcha')
            cp.send_keys(captcha)
            self.wd.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/div/div/form/input[2]').click()
        except:
            return "error"

In [30]:
class Refresh():
    def WebScrap(self):
        try:
            body = self.wd.find_element_by_tag_name("body")
            body.send_keys(Keys.ALT + Keys.ARROW_LEFT)
            nextPage = driver.find_element_by_xpath('//*[@id="todo-conteudo-resultado"]/div/div/center/div/ul/li[7]/a').get_attribute("href")
            self.driver.get(nextPage)
        except:
            return "error"

In [69]:
class ReclameScrap:
    
    def __init__(self):
        
        db = Database('reclameAqui_db.db')
        sqlite = Sqlite(db)
        self.dao = Dao(sqlite)
        self.tables = self.dao.getTables()
        if (u'log',) not in self.tables:
            self.dao.createTable("log","(log_ID INTEGER PRIMARY KEY AUTOINCREMENT, time TEXT, tableName TEXT,"
                                 "business TEXT, quant INTEGER,lastIn_ID)")
        else:
            self.cleanTable()
    
    def execute(self):
        
        if (u'complaint',) in self.tables:
            dao = ComplaintDao(sqlite)
            scrap = ScrapComplaint(dao)
        elif (u'address',) in self.tables:
            dao = AddressDao(sqlite)
            scrap = ScrapAddress(dao)
        else:
            dao = RankDao(sqlite)
            dao.createTable()
            scrap = ScrapRank(dao)
    
    def cleanTable(self):  
        logVal = self.dao.putQuery("SELECT tabela, business FROM log"
                                    " WHERE log_id = (SELECT MAX(rank_id) FROM log);")
        if logVal[0][0] is "address":
            self.dao.putQuery("DELETE FROM address WHERE business = "+logVal[0][1]+";")

In [32]:
db = Database('teste.db')
sql = Sqlite(db)
dao = Dao(sql)

In [33]:
dao.getTables()

[(u'rank',), (u'sqlite_sequence',)]

In [34]:
dao.putQuery("SELECT * FROM sqlite_sequence;")

[(u'rank', 7)]

In [35]:
dao = RankDao(sql)
dao.createTable()

Could not create the table


In [82]:
comp = Complaint()
comp.business = 'teste675'
comp.businessURL = 'www.showke'
dao.insert(comp)

In [88]:
a = dao.putQuery("SELECT MAX(rank_ID) FROM rank;")
a

[(10,)]

In [38]:
a = dao.putQuery("SELECT rank_id FROM rank"
                " GROUP BY rank_id HAVING rank_id <2;")
for i in a:
    print i
    

(1,)


In [39]:
cond = "o macaco = bola"

In [40]:
terms = cond.split()
cond = "GROUP BY "+terms[0]+" "+cond
print cond

GROUP BY o o macaco = bola


In [41]:
"bola"+str(None)

'bolaNone'

In [42]:
obj = np.empty((0,0))

In [43]:
np.append([3,2],obj)

array([ 3.,  2.])

In [44]:
for i in range(3):
    print i

0
1
2


In [95]:
ar = [4]

In [96]:
ar[0] = datetime.utcnow().ctime()

'Sun May  3 18:39:11 2015'